# Data feed (optional)

Using a notebook to feed data into Event Store is now optional for this code pattern.
Instead, you should use IBM Streams to feed the data. If you are not using IBM Streams,
you can run this notebook as an alternative.

In this notebook, a CSV file is read into a pandas DataFrame
which is then used to batch insert the rows into Event Store.

Before running this notebook, you need to run the **Event Store Table Creation**
notebook to setup the database and table.

## Connect to IBM Db2 Event Store

Edit the values in the next cell

In [ ]:
CONNECTION_ENDPOINT=""

EVENT_USER_ID=""

EVENT_PASSWORD=""

# Port will be 1100 for version 1.1.2 or later (5555 for version 1.1.1)
PORT = "30370"

DEPLOYMENT_ID=""

# Database name
DB_NAME = "EVENTDB"

# Table name
TABLE_NAME = "IOT_TEMPERATURE"

HOSTNAME=""

DEPLOYMENT_SPACE=""

In [ ]:

bearerToken=!echo `curl --silent -k -X GET https://{HOSTNAME}:443/v1/preauth/validateAuth -u admin:password |python -c "import sys, json; print(json.load(sys.stdin)['accessToken'])"`
bearerToken=bearerToken[0]
keystorePassword=!echo `curl -k --silent  GET -H "authorization: Bearer {bearerToken}" "https://{HOSTNAME}:443/icp4data-databases/{DEPLOYMENT_ID}/zen/com/ibm/event/api/v1/oltp/keystore_password"`
keystorePassword

## Import Python modules


In [ ]:
## Note: Only run this cell if your IBM Db2 Event Store is installed with IBM Cloud Pak for Data (CP4D)

# In IBM Cloud Pak for Data, we need to create link to ensure Event Store Python library is 
# properly exposed to the Spark runtime.
import os
src = '/home/spark/user_home/eventstore/eventstore'
dst = '/home/spark/shared/user-libs/python3.6/eventstore'
try:
    os.remove(dst)
except EnvironmentError as e:
    print("Symlink doesn't exist, creating symlink to include Event Store Python library...")
os.symlink(src, dst)
print("Creating symlink to include Event Store Python library...")

In [ ]:
from eventstore.common import ConfigurationReader
from eventstore.oltp import EventContext

## Connect to Event Store

In [ ]:
ConfigurationReader.setConnectionEndpoints(CONNECTION_ENDPOINT)
ConfigurationReader.setEventUser(EVENT_USER_ID)
ConfigurationReader.setEventPassword(EVENT_PASSWORD)
ConfigurationReader.setSslKeyAndTrustStorePasswords(keystorePassword[0])
ConfigurationReader.setDeploymentID(DEPLOYMENT_ID)
ConfigurationReader.getSslTrustStorePassword()

## Load the tables and inspect the table schemas

The following cell gets the schema of the table.

In [ ]:
ctx = EventContext.get_event_context(DB_NAME)
resolved_table_schema = ctx.get_table(TABLE_NAME)
print(resolved_table_schema)

## Load the sample data

Let's insert some sample data into the table before starting analysis.
To load data into the table, you will have to:

- Use the Event Store UI to add the "sample_IOT_table.csv" file as a project data asset.
- Run the following cells to load the data from **assets**.

### Read the CSV into a pandas DataFrame

In [ ]:

import pandas as pd

df = pd.read_csv('/project_data/data_asset/sample_IOT_table.csv', names=['DEVICEID','SENSORID','TS','AMBIENT_TEMP','POWER','TEMPERATURE'])
df.head()


### Examine the data types

In [ ]:
df.dtypes

In [ ]:
df.to_dict('records')

### Batch insert the data from the DataFrame into the table

In [ ]:
ctx.batch_insert(resolved_table_schema, df.to_dict('records'))

## Summary
In this notebook, you learned how to:
- Insert data into a table from a CSV file

<p><font size=-1 color=gray>
&copy; Copyright 2019 IBM Corp. All Rights Reserved.
<p>
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file
except in compliance with the License. You may obtain a copy of the License at
https://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the
License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either
express or implied. See the License for the specific language governing permissions and
limitations under the License.
</font></p>